# Reto CDAW: Loan Approval Prediction in Penslyvania

## Imports

In [1]:
import pandas as pd
from pandas import Series, DataFrame

# Training and test spliting
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Estimators
from sklearn.svm import SVC

# Evaluation
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report, recall_score, precision_score, make_scorer
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import sem

# Decision tree alorithm
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# K neighbors classifier
from sklearn.neighbors import KNeighborsClassifier as kNC

# Optimization
from sklearn.model_selection import GridSearchCV

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
sns.set(color_codes=True)

## Data Understanding

In [2]:
df = pd.read_csv('/kaggle/input/loanapprovalcdaw-traintest/train.csv')
df = pd.DataFrame(df)
df [:5]

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,1-Nov-05,2006,2,...,0,2,0,1,N,N,31-Dec-05,"$8,000.00",$0.00,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,6-Jun-05,2005,2,...,1,2,1,1,N,N,31-Jul-05,"$166,000.00",$0.00,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,18-Mar-03,2003,2,...,4,2,1,2,Y,N,31-Mar-03,"$25,000.00",$0.00,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,28-Jun-95,1995,2,...,0,0,1,0,N,N,31-Jan-96,"$220,100.00",$0.00,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,16-Apr-09,2009,0,...,0,0,0,1,N,N,31-May-09,"$25,000.00",$0.00,0


In [3]:
df.info

<bound method DataFrame.info of                 id  LoanNr_ChkDgt                            Name        City  \
0      bd9d6267ec5     1523195006        P-SCAPE LAND DESIGN, LLC  NORTHFIELD   
1      9eebf6d8098     1326365010  The Fresh & Healthy Catering C      CANTON   
2      83806858500     6179584001          AARON MASON & HOWE LLC  SAWYERWOOD   
3      a21ab9cb3af     8463493009               MID OHIO CAR WASH    COLUMBUS   
4      883b5e5385e     3382225007                 Bake N Brew LLC      Newark   
...            ...            ...                             ...         ...   
22830  4f9443d2a46     1573725008    SIBILA RACE ENGINEERING, INC   MASSILLON   
22831  798db2753a7     2011184008     ENVIRO SHIELD POWER WASHING  SPRINGBORO   
22832  ddb3c5e9bff     4082983001  MAINLINE TRCK&TRAILR SRVC, INC     BEDFORD   
22833  407200a5dfe     7783283010                  TIN BOX STUDIO  CINCINNATI   
22834  eaa66193392     1513375008                     Lowex, Inc.      Dayton

In [4]:
df.dtypes

id                    object
LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Bank                  object
BankState             object
ApprovalDate          object
ApprovalFY             int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
Accept                 int64
dtype: object

In [5]:
print(df.shape)

print(df.groupby("Accept").size())

(22835, 21)
Accept
0     3831
1    19004
dtype: int64


## Data preparation

### Type conversion

In [6]:
# Converting all strings to LowerCase
df = df.map(lambda x: x.lower() if isinstance(x, str) else x)
df[:5]

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"p-scape land design, llc",northfield,oh,citizens bank natl assoc,ri,1-nov-05,2006,2,...,0,2,0,1,n,n,31-dec-05,"$8,000.00",$0.00,1
1,9eebf6d8098,1326365010,the fresh & healthy catering c,canton,oh,"firstmerit bank, n.a.",oh,6-jun-05,2005,2,...,1,2,1,1,n,n,31-jul-05,"$166,000.00",$0.00,1
2,83806858500,6179584001,aaron mason & howe llc,sawyerwood,oh,"pnc bank, national association",oh,18-mar-03,2003,2,...,4,2,1,2,y,n,31-mar-03,"$25,000.00",$0.00,1
3,a21ab9cb3af,8463493009,mid ohio car wash,columbus,oh,the huntington national bank,oh,28-jun-95,1995,2,...,0,0,1,0,n,n,31-jan-96,"$220,100.00",$0.00,1
4,883b5e5385e,3382225007,bake n brew llc,newark,oh,the huntington national bank,oh,16-apr-09,2009,0,...,0,0,0,1,n,n,31-may-09,"$25,000.00",$0.00,0


In [7]:
# convert dates "str" to "datetime64" for better operability
df["DisbursementDate"]= pd.to_datetime(df["DisbursementDate"], format="mixed")
df["ApprovalDate"]= pd.to_datetime(df["ApprovalDate"], format="mixed")

print(df["DisbursementDate"][:5])
print(df["ApprovalDate"][:5])

0   2005-12-31
1   2005-07-31
2   2003-03-31
3   1996-01-31
4   2009-05-31
Name: DisbursementDate, dtype: datetime64[ns]
0   2005-11-01
1   2005-06-06
2   2003-03-18
3   1995-06-28
4   2009-04-16
Name: ApprovalDate, dtype: datetime64[ns]


In [8]:
# Convert object of type $XXX,XXX.XX to integers to be able to operate with them
df['DisbursementGross'] = (
    df['DisbursementGross']
    .astype(str)
    .str.replace(r'[$,]', '', regex=True)  # Remove $ and ,
    .str.strip()  # Remove extra spaces
    .astype(float)  # Convert to float
)

df['BalanceGross'] = (
    df['BalanceGross']
    .astype(str)
    .str.replace(r'[$,]', '', regex=True)  # Remove $ and ,
    .str.strip()  # Remove extra spaces
    .astype(float)  # Convert to float
)

In [9]:
## Convertthe Revolving line of Credit to Bool
df['RevLineCr'] = df['RevLineCr'].map({'y': True, 'n': False})
df['RevLineCr']

0        False
1        False
2         True
3        False
4        False
         ...  
22830    False
22831    False
22832    False
22833    False
22834    False
Name: RevLineCr, Length: 22835, dtype: object

In [18]:
## Convert the LowDoc to boolean
df['LowDoc'] = df['LowDoc'].map({'y': True, 'n': False})
df['LowDoc']

0        False
1        False
2        False
3        False
4        False
         ...  
22830    False
22831     True
22832    False
22833     True
22834    False
Name: LowDoc, Length: 22813, dtype: object

In [10]:
df.dtypes

id                           object
LoanNr_ChkDgt                 int64
Name                         object
City                         object
State                        object
Bank                         object
BankState                    object
ApprovalDate         datetime64[ns]
ApprovalFY                    int64
NoEmp                         int64
NewExist                    float64
CreateJob                     int64
RetainedJob                   int64
FranchiseCode                 int64
UrbanRural                    int64
RevLineCr                    object
LowDoc                       object
DisbursementDate     datetime64[ns]
DisbursementGross           float64
BalanceGross                float64
Accept                        int64
dtype: object

### Duplicated, null and missing values

In [11]:
# chek if there are any duplicated entries or missing values.
print("Duplicated entries: ",df.duplicated().any(),"\n")
missingValues = df.isna().sum()
print("Missing values per feature: \n", missingValues)

Duplicated entries:  False 

Missing values per feature: 
 id                      0
LoanNr_ChkDgt           0
Name                    1
City                    1
State                   0
Bank                   22
BankState              22
ApprovalDate            0
ApprovalFY              0
NoEmp                   0
NewExist                2
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            5583
LowDoc                 43
DisbursementDate       61
DisbursementGross       0
BalanceGross            0
Accept                  0
dtype: int64


First of all lets see whether the entries with any missing data correspond to loans that ahve been granted or denied.

This is interesting since the dataset is unbalanced.

In [12]:
empty = df.loc[
    (df.isna().any(axis=1)),
    'id'].size
granted_empty = df.loc[
    (df.isna().any(axis=1)) &
    (df.Accept == 1),
    'id'].size


print("Total entries with mising values: ", empty)
print("Entries with missing data that were granted a loan: ", granted_empty)

Total entries with mising values:  5709
Entries with missing data that were granted a loan:  4873


Si the vast majority of the entries were granted a loan, which is the majority class of our unbalanced dataset, a solution would be to delete all the entries with missing values.

This would help us balnce the dataset as well to clean the data.

Nontheless, lets try to minimize the deleted data!

In [13]:
## Add/Remove missing data entries
df[df.City.isna()]
## As we can see there is only one entry where the city is NaN, we can asume the city to be Lakewood from the name of the company.


,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
20014,3ed3b653739,3258222000,lakewood health care center lt,NaN,oh,growth capital corp.,oh,1981-11-18,1982,40,...,0,0,0,0,False,n,1982-02-10,205000.0,0.0,1


In [14]:
## We set the city to be lakewood
# df['City'].fillna('lakewood', inplace=True)
df.fillna({'City': 'lakewoood'}, inplace=True)
df[df['id']=='3ed3b653739']

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
20014,3ed3b653739,3258222000,lakewood health care center lt,lakewoood,oh,growth capital corp.,oh,1981-11-18,1982,40,...,0,0,0,0,False,n,1982-02-10,205000.0,0.0,1


Since we were able to fill the missing data of this entry, **there is no need to delete it.**

In [15]:
## For the case of the missing name:
df[df.Name.isna()]

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
1085,add8fbfb491,1616875008,NaN,cleveland,oh,"firstmerit bank, n.a.",oh,2006-01-17,2006,1,...,1,0,0,1,NaN,n,2006-07-31,22044.0,0.0,1


Since the name of the company is not considered an **important feature** it will not be added to the features list used to train the model.

Therefore, there is **no need to delete this entry.**


For the entries with **missingBank Name** we will procede to its elimination. 

In [16]:
df = df.dropna(subset=['Bank'])
missingValues = df.isna().sum()
print("Missing values per feature: \n", missingValues)

Missing values per feature: 
 id                      0
LoanNr_ChkDgt           0
Name                    1
City                    0
State                   0
Bank                    0
BankState               0
ApprovalDate            0
ApprovalFY              0
NoEmp                   0
NewExist                2
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            5583
LowDoc                 43
DisbursementDate       60
DisbursementGross       0
BalanceGross            0
Accept                  0
dtype: int64


## Feature Definition

In [23]:
df[df["State"]!= 'oh']

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept


As we can see, the State does not provide any information since all the entries share the same value. Therefore it will not be added into the features list.

In [ ]:
x_train = [df[--],] 
y_train = df["Accept"]